In [2]:
import cifar10_input
import tensorflow as tf
import numpy as np

W1118 19:39:46.946985 140219890218752 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
batch_size = 128
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'
print("begin")
images_train, labels_train = cifar10_input.inputs(eval_data = False, batch_size = batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data = True, batch_size = batch_size)
print("begin data")

begin


/home/lx/.pyenv/versions/3.7.3/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


W1118 19:55:10.000881 140219890218752 deprecation.py:323] From /home/lx/.local/lib/python3.7/site-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


W1118 19:55:27.603504 140219890218752 dataset_builder.py:862] Statistics not present in the dataset. TFDS is not able to load the total number of examples, so using the subsplit API may not provide precise subsplits.


W1118 19:55:36.353295 140219890218752 deprecation.py:323] From /home/lx/.local/lib/python3.7/site-packages/tensorflow_datasets/core/dataset_info.py:548: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
W1118 19:55:36.358492 140219890218752 deprecation.py:323] From /home/lx/.local/lib/python3.7/site-packages/tensorflow_datasets/core/dataset_info.py:549: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
W1118 19:55:36.362120 140219890218752 dataset_builder.py:862] Statistics not present in the dataset. TFDS is not able to load the total number of examples, so using the subsplit API may not provide precise subsplits.



Dataset cifar10 downloaded and prepared to /home/lx/tensorflow_datasets/cifar10/1.0.2. Subsequent calls will reuse this data.


W1118 19:56:14.762300 140219890218752 deprecation_wrapper.py:119] From /home/lx/learn_tf/level_1/cifar10_input.py:75: The name tf.image.resize_image_with_crop_or_pad is deprecated. Please use tf.image.resize_with_crop_or_pad instead.

W1118 19:56:14.841619 140219890218752 deprecation.py:323] From /home/lx/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:1514: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W1118 19:56:14.893741 140219890218752 deprecation.py:323] From /home/lx/learn_tf/level_1/cifar10_input.py:45: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last 

begin data


In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
  
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')  
                        
def avg_pool_6x6(x):
  return tf.nn.avg_pool(x, ksize=[1, 6, 6, 1],
                        strides=[1, 6, 6, 1], padding='SAME')

In [5]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, 24,24,3]) # cifar data image of shape 24*24*3
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 数字=> 10 classes


W_conv1 = weight_variable([5, 5, 3, 64])
b_conv1 = bias_variable([64])

x_image = tf.reshape(x, [-1,24,24,3])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 64, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_conv3 = weight_variable([5, 5, 64, 10])
b_conv3 = bias_variable([10])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)

nt_hpool3=avg_pool_6x6(h_conv3)#10
nt_hpool3_flat = tf.reshape(nt_hpool3, [-1, 10])
y_conv=tf.nn.softmax(nt_hpool3_flat)

cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners(sess=sess)
for i in range(15000):#20000
  image_batch, label_batch = sess.run([images_train, labels_train])
  label_b = np.eye(10,dtype=float)[label_batch] #one hot
  
  train_step.run(feed_dict={x:image_batch, y: label_b},session=sess)
  
  if i%200 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:image_batch, y: label_b},session=sess)
    print( "step %d, training accuracy %g"%(i, train_accuracy))


image_batch, label_batch = sess.run([images_test, labels_test])
label_b = np.eye(10,dtype=float)[label_batch]#one hot
print ("finished！ test accuracy %g"%accuracy.eval(feed_dict={
     x:image_batch, y: label_b},session=sess))

sess.close()

W1118 19:56:15.882279 140219890218752 deprecation.py:323] From <ipython-input-6-b66f1a294503>:3: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1118 19:56:15.883253 140219890218752 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


step 0, training accuracy 0.109375
step 200, training accuracy 0.3125
step 400, training accuracy 0.367188
step 600, training accuracy 0.453125
step 800, training accuracy 0.414062
step 1000, training accuracy 0.453125
step 1200, training accuracy 0.476562
step 1400, training accuracy 0.515625
step 1600, training accuracy 0.53125
step 1800, training accuracy 0.484375
step 2000, training accuracy 0.484375
step 2200, training accuracy 0.484375
step 2400, training accuracy 0.59375
step 2600, training accuracy 0.632812
step 2800, training accuracy 0.5
step 3000, training accuracy 0.5625
step 3200, training accuracy 0.625
step 3400, training accuracy 0.554688
step 3600, training accuracy 0.554688
step 3800, training accuracy 0.648438
step 4000, training accuracy 0.492188
step 4200, training accuracy 0.492188
step 4400, training accuracy 0.546875
step 4600, training accuracy 0.65625
step 4800, training accuracy 0.6875
step 5000, training accuracy 0.546875
step 5200, training accuracy 0.52343

KeyboardInterrupt: 

In [ ]:
# 启动session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(200):#20000
      batch = mnist.train.next_batch(50)#50
      if i%20 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y: batch[1]})
        print( "step %d, training accuracy %g"%(i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y: batch[1]})
    
    print ("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y: mnist.test.labels}))